In [1]:
from pathlib import Path
import warnings

import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Local imports
from mlp import MLP, MLPDataset, train_test_ae_mlp


warnings.filterwarnings("ignore", category=RuntimeWarning)


# Constants
BATCH_SIZE = 1028
DATA_DIR = Path(r"C:\Users\Peter\gh\rasmussen-705.603\data\FinalProject")
EPOCHS = 30
GAMMA = 0.1
LAT_SPACE_SIZES = [2, 4, 16, 64, 256,]
LRS = [0.5, 0.1]
OUTPUT_BANDS = ["blue", "green", "red", "nir08", "swir16", "swir22", "ndvi", "qa"]
STEP_SIZES = [5, 10, 15]
TR_VAL_SPLIT = 0.9
WEIGHT_DECAY = 5e-4

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define directories
raw_dir = DATA_DIR / "raw"
interim_dir = DATA_DIR / "interim"
processed_dir = DATA_DIR / "processed"
cogs_dir = interim_dir / "cogs"
autoencoder_dir = processed_dir / "autoencoder"
models_dir = interim_dir / "models"
scores_dir = interim_dir / "scores"
models_dir.mkdir(exist_ok=True, parents=True)
scores_dir.mkdir(exist_ok=True, parents=True)

In [2]:
# Create PyTorch datasets and loaders
dataset = MLPDataset(cogs_dir, OUTPUT_BANDS, region="", train=True, slice_frac=0.01)

tr_val_size = int(TR_VAL_SPLIT * len(dataset))
te_size = len(dataset) - tr_val_size
tr_size = int(TR_VAL_SPLIT * tr_val_size)
val_size = tr_val_size - tr_size

tr_val_dataset, te_dataset = torch.utils.data.random_split(dataset, [tr_val_size, te_size])
tr_dataset, val_dataset = torch.utils.data.random_split(tr_val_dataset, [tr_size, val_size])

tr_loader = DataLoader(tr_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
te_loader = DataLoader(te_dataset, batch_size=BATCH_SIZE, shuffle=False)


len(tr_loader), len(val_loader), len(te_loader)

(1738, 194, 215)

In [3]:
from mlp.datasets import *
cogs_meta = dataset.cogs_meta.copy()
slice_frac = 0.01
train = True

In [7]:
epochs = EPOCHS
_, in_features = dataset.cube.shape
tr_val_losses = []
te_losses = []
for step_size in STEP_SIZES:
    for lr in LRS:

        print(f"Train model with step_size={step_size} and lr={lr}.")
        model = MLP(
            in_features=in_features,
            hidden_neurons=2,
            )
        model.to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=WEIGHT_DECAY, fused=True)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=GAMMA)
        tr_val_losses_, te_losses_ = train_test_ae_mlp(
            model,
            tr_loader, val_loader, te_loader,
            criterion, optimizer, scheduler,
            lr, step_size, epochs,
            models_dir, scores_dir,
            device,
        )
        tr_val_losses.extend(tr_val_losses_)
        te_losses.extend(te_losses_)

tr_val_summary = pd.DataFrame(tr_val_losses)
te_summary = pd.DataFrame(te_losses)

tr_val_summary.to_csv(scores_dir / "tr_val_summary.csv")
te_summary.to_csv(scores_dir / "te_summary.csv")

Train model with step_size=5 and lr=0.5.
[5][0.500][1]: tr_loss_per_batch=0.3279
[5][0.500][1]: val_loss_per_batch=0.3535
[5][0.500][1]: batch_val_loss=0.3535, lr=0.500000
[5][0.500][1]: Saved model b/c val_loss at all-time low.
[5][0.500][2]: tr_loss_per_batch=0.3413
[5][0.500][2]: val_loss_per_batch=0.4419
[5][0.500][2]: batch_val_loss=0.4419, lr=0.500000
[5][0.500][2]: Did not save model b/c val_loss too high.
[5][0.500][3]: tr_loss_per_batch=0.3468
[5][0.500][3]: val_loss_per_batch=0.2918
[5][0.500][3]: batch_val_loss=0.2918, lr=0.500000
[5][0.500][3]: Saved model b/c val_loss at all-time low.
[5][0.500][4]: tr_loss_per_batch=0.3133
[5][0.500][4]: val_loss_per_batch=0.2860
[5][0.500][4]: batch_val_loss=0.2860, lr=0.500000
[5][0.500][4]: Saved model b/c val_loss at all-time low.
[5][0.500][5]: tr_loss_per_batch=0.3137
[5][0.500][5]: val_loss_per_batch=0.3139
[5][0.500][5]: batch_val_loss=0.3139, lr=0.500000
[5][0.500][5]: Did not save model b/c val_loss too high.
[5][0.500][6]: tr_l